In [1]:
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import os
import json
import torch
import transformers
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_df = pd.read_csv('./datasets/total.csv')
dataset_df = dataset_df.drop(axis=1, columns=['Unnamed: 0'])
print(dataset_df.head())
dataset_custom = Dataset.from_pandas(dataset_df)
print(dataset_custom)

                    instruction                                output
0                    잘 자고 일어났니?                    응, 잘 잤어. 엄마는 어떠셨어?
1                      아침은 먹었어?              네, 간단하게 먹었어. 엄마 아침은 드셨어?
2                최근에 좋은 일이 있었니?             응, 지난주에 친구들하고 즐거운 시간 보냈어.
3  날씨가 많이 추워졌어. 따뜻하게 입고 다니고 있니?  응, 그래서 두꺼운 옷으로 갈아입었어. 엄마도 따뜻하게 입으세요.
4               오늘 저녁엔 뭐 먹고 싶어?            한식이 먹고 싶어. 엄마는 뭐 드시고 싶으세요?
Dataset({
    features: ['instruction', 'output'],
    num_rows: 2035
})


In [3]:
dataset_custom.save_to_disk("dataset/dataset")

Saving the dataset (1/1 shards): 100%|██████████| 2035/2035 [00:00<00:00, 405521.13 examples/s]


In [4]:
BASE_MODEL = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    # load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    load_in_8bit=True, # 모델 학습 시간 및 gpu 자원 제한으로 8비트 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=nf4_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.79s/it]


In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [6]:
dataset_path = "dataset/dataset"
dataset = load_from_disk(dataset_path)

In [7]:
prompt_input_template = """아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 입력:
{input}

### 응답:"""


prompt_no_input_template = """아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 응답:"""

In [10]:
def generate_prompt(data_point):
  instruction = data_point["instruction"]
  # input = data_point["input"]
  label = data_point["output"]

  # if input:
  #   res = prompt_input_template.format(instruction=instruction, input=input)
  # else:
  #   res = prompt_no_input_template.format(instruction=instruction)
  
  res = prompt_no_input_template.format(instruction=instruction)

  if label:
    res = f"{res}{label}<|im_end|>" # eos_token을 마지막에 추가

  data_point['text'] = res

  return data_point

In [11]:
# 데이터셋에 프롬프트 적용
remove_column_keys = dataset.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = dataset.shuffle().map(generate_prompt, remove_columns=remove_column_keys)

Map: 100%|██████████| 2035/2035 [00:00<00:00, 14238.88 examples/s]


In [12]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs

In [13]:
remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map: 100%|██████████| 2035/2035 [00:00<00:00, 13537.03 examples/s]


In [14]:
lora_config = LoraConfig(
    r=4, # LoRA 가중치 행렬의 rank. 정수형이며 값이 작을수록 trainable parameter가 적어짐
    lora_alpha=8, # LoRA 스케일링 팩터. 추론 시 PLM weight와 합칠 때 LoRA weight의 스케일을 일정하게 유지하기 위해 사용
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], # LoRA를 적용할 layer. 모델 아키텍처에 따라 달라짐
    bias='none', # bias 파라미터를 학습시킬지 지정. ['none', 'all', 'lora_only']
    task_type=TaskType.CAUSAL_LM
)

# 양자화된 모델을 학습하기 전, 전처리를 위해 호출
model = prepare_model_for_kbit_training(model)
# LoRA 학습을 위해서는 아래와 같이 peft를 사용하여 모델을 wrapping 해주어야 함
model = get_peft_model(model, lora_config)

# 학습 파라미터 확인
model.print_trainable_parameters()

trainable params: 10,485,760 || all params: 8,040,747,008 || trainable%: 0.1304


In [15]:
# Data Collator 역할
# 각 입력 시퀀스의 input_ids(토큰) 길이를 계산하고, 가장 긴 길이를 기준으로 길이가 짧은 시퀀스에는 패딩 토큰 추가
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [16]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=2, # 각 디바이스당 배치 사이즈. 작을수록(1~2) 좀 더 빠르게 alignment 됨
    gradient_accumulation_steps=4, 
    warmup_steps=1,
    #num_train_epochs=1,
    max_steps=1000, 
    learning_rate=2e-4, # 학습률
    bf16=True, # bf16 사용 (지원되는 하드웨어 확인 필요)
    output_dir="outputs",
    optim="paged_adamw_8bit", # 8비트 AdamW 옵티마이저
    logging_steps=50, # 로깅 빈도
    save_total_limit=3 # 저장할 체크포인트의 최대 수
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    max_seq_length=512, # 최대 시퀀스 길이
    args=train_args,
    dataset_text_field="text",
    data_collator=collate_fn
)

/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning

In [17]:
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to f

Step,Training Loss
50,1.243000
100,0.791700
150,0.777700
200,0.767900
250,0.732600
300,0.631700
350,0.617900
400,0.617100
450,0.601200
500,0.603900


/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
 

TrainOutput(global_step=1000, training_loss=0.5863430480957031, metrics={'train_runtime': 10599.9691, 'train_samples_per_second': 0.755, 'train_steps_per_second': 0.094, 'total_flos': 2.8585149163708416e+16, 'train_loss': 0.5863430480957031, 'epoch': 3.9292730844793713})

In [18]:
FINETUNED_MODEL = "pcp_qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)